# Implementation of Meshroom and PCL_based labeller

Makes use of the local programs:
- run_meshroom.py
- preprocessing.py
- pcl_imp.py
- utils.py
- mesh_cropper.py
- pcl_imp.exe



In [2]:
from run_meshroom import run_meshroom
from preprocessing import Scene
from pcl_imp import pcl_imp
from mesh_cropper import mesh_cropper

import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# MESHROOM MESH CREATION

Python code to run Meshroom on the command line. Go to the run_meshroom.py or use help command for more information about the function. A base directory is used. 

Note: Meshroom is often stopped by the antivirus, provide the necessary permissions for the .exe files in the meshroom directory. (..\Meshroom-2021.1.0\aliceVision\bin)

In [ ]:
help(run_meshroom)

In [ ]:
def meshroom_mod(base_dir):
    images_name = 'images'
    graph_name = 'graph_1.mg'
    cache_name = 'MeshroomCache'
    
    ## Compute Mesh from meshroom graph pipeline.
    
    scene_base_dir = run_meshroom(base_dir,images_name, graph_name, cache_name)
    
    print('The mesh and pcd were created and stored at ' + scene_base_dir)
    
    return scene_base_dir

# OPEN 3D BASED PREPROCESSING
The mesh and pcd are loaded and then preprocessed using the local Scene class. Uncomment display to show both the mesh and pcd. Go to preprocessing.py for more information on which methods are applied or use help command. Mesh and pcd must be in the same directory.

In [ ]:
help(Scene)

In [ ]:
def preprocessing_mod(scene_base_dir):
    # Load pcd and mesh 
    pcd_dir = scene_base_dir + r'\sfm.ply'
    mesh_dir = scene_base_dir + r'\mesh.obj'    
    pcd = o3d.io.read_point_cloud(pcd_dir)
    mesh = o3d.io.read_triangle_mesh(mesh_dir,enable_post_processing=True)
    
    #Init Scene class
    scene = Scene(mesh, pcd, scene_base_dir)
    
    # Preprocessing
    scene.pcd_outlier_removal(nb_neighbors=100, std_ratio=0.6)
    scene.floor_alignment()
    scene.crop_dim()
    scene.pcd_outlier_removal(nb_neighbors=100, std_ratio=0.6)
    scene.pcd_outlier_removal(nb_neighbors=100, std_ratio=0.6)
    scene.x_alignment(threshold = 0.05)
    
    #Verbosity
    # scene.display_pcd()
    # scene.display_mesh()
    # utils.display_geo([scene.mesh,scene.pcd])
   
    #Save processed mesh and pcd, return corresponding directories
    processed_mesh_dir = scene.save_mesh()
    processed_pcd_dir = scene.save_pcd()
    downsampled_processed_pcd_dir = scene.save_down_pcd(voxel_size = 0.02)
    
    return processed_mesh_dir, processed_pcd_dir, downsampled_processed_pcd_dir

# PCL BASED SEGMENTATION AND LABELLING
The processed pcd is loaded to the C++ based labelling algoritm.
This process only makes use of the pcd.

In [ ]:
help(pcl_imp)

In [ ]:
def pcl_mod(processed_pcd_dir):
    #Executable directory
    exe_dir = r'C:\Users\lfcas\Documents\Internship\Final\PCL\Debug\pcl_imp.exe'
    
    # defining parameters. If left at 0 the defaults will be chosen
    dt = 0
    pct = 0
    rct = 0
    min_cs = 0
    sigma = 0
    radius = 0
    nn = 0
    sw = 0
    
    #Setting up Vectors of parameters
    color_RGS = [dt, pct, rct, min_cs]
    min_cut = [sigma, radius, nn, sw]
    
    csv_dir = pcl_imp(exe_dir, processed_pcd_dir, color_RGS, min_cut)
    
    return csv_dir


# Mesh Cropper
Crops meshes by given indices list from a .csv file, labels them and stores new meshes. The first column of the csv file represents the label, the rest are a list of point indices representing the cloud. The mesh cropper module has no hyperparameters, for more info go to mesh_cropper.py or use help function. It implements a standard statistical outlier removal that can be control outside the .py (use help() to know more).


In [3]:
help(mesh_cropper)

Help on function mesh_cropper in module mesh_cropper:

mesh_cropper(mesh_dir, pcd_dir, csv_dir, nb_neighbors=100, std_ratio=0.6)
    Mesh Cropper
    Crops meshes by given indices list, labels them and stores new mesh in .csv base directory.
    
    Parameters
    ----------
    mesh_dir : STRING
        directory of mesh
    pcd_dir : STRING
        directory of point cloud
    csv_dir : STRING
        directory of .csv file
    nb_neighbors : INT
        number of neighbors for statistical outlier removal. default = 100
    std_ratio : Float
        Standard ratio for statistical outlier removal
    
    Returns
    -------
    None.



# Modules Implementation
Once you have fixed the hyperparameters run the next cell to implement the whole code. You can limit the amount of modules you use depending on the intended outcome. For example: 
- If the intention is to create the mesh just use the meshroom module.
- If the intention is to process existing mesh and pcd, only use the preprocessing module. Provide directly the required directories with the allowed file formats.
- The pcl can also be implemented directly without preprocessing the mesh. Be aware that this most likely will require fine tunning the pcl arguments (use help(pcl_imp.py))

In [ ]:
    base_dir = r'C:\Users\lfcas\Documents\Internship\Final\\'
    #base_dir: base directory where Meshroom program, meshroom .mg graph file, cache folder and output folder are located.
    
    sbd = meshroom_mod(base_dir)
    
    #sbd = r'C:\Users\lfcas\Documents\Internship\Final\Meshes\1646649959'
    #sbd: scene base directory
    
    
    pmd, ppd, dppd = preprocessing_mod(sbd)
    
    #pmd: processed mesh directory
    #ppd: processed pcd directory
    #dppd: downsampled processed pcd directory
    
    #dppd = r'C:\Users\lfcas\Documents\Internship\Final\Meshes\1646649959\processed_pcd.pcd'
    
    csv_dir = pcl_mod(ppd)
    
    #csv_dir: .csv file directory
    
    
    mesh_cropper(pmd, dppd, csv_dir)
    
    